In [ ]:
!pip install pandas
!pip install petl
!pip install openpyxl
!pip install glob



In [ ]:
import pandas as pd
import petl as etl
import glob, os, json



In [ ]:
#check CSV dfs
#product_data =  etl.fromcsv('product_data/2010-2020_Bassetlaw_outlet.csv')
#product_data.look()



In [ ]:
#check json dfs
#json_table = etl.fromjson('json_table/2010-2020_Ceredigion_store.json')
#json_table.look()

In [ ]:
#concatenate all CSV files 
path = "data/original_product_data/"

all_files = glob.glob(os.path.join(path, "*.csv"))
all_df = []
for f in all_files:
    print ("...")
    df = pd.read_csv(f, sep=',')
    #2010-2020_Bassetlaw_outlet.csv
    file_name= " ".join(f.split('/')[-1].split("_")[1:]).replace(".csv", "").strip()
    df['branch'] = file_name
    all_df.append(df)
    
        
merged_df = pd.concat(all_df, ignore_index=True, sort=True)
merged_df

In [ ]:
merged_df.to_csv("data/csv_product_data.csv")

In [ ]:
path = 'data/json_files_dir/'

file_list = glob.glob(os.path.join(path, "*.json"))
print(file_list)

dfjson = []
for file in file_list:
    
    print("...")
    json_data = pd.read_json(file)
    file_name= " ".join(file.split('/')[-1].split("_")[1:]).replace(".json", "").strip()
    json_data['branch'] = file_name 
    
    dfjson.append(json_data)
    
merge_dfjson = pd.concat(dfjson, ignore_index=True, sort=True)
(merge_dfjson)


In [ ]:
#export to csv single file
merge_dfjson.to_csv("json_product_data.csv")

In [ ]:
#inspect excel branch expenses data and create table
expenses_db_table_excel = etl.fromxlsx('data/branch_expenses.xlsx')
expenses_db_table_excel.look()

In [ ]:
#inspect excel branch list data and create table
blist_db_table_excel=etl.fromxlsx('data/branch_list.xlsx')
blist_db_table_excel.look()

In [ ]:
#inspect product list data and create table
product_list = etl.fromcsv('data/original_product_data/products_list.csv')
product_list.look()

In [ ]:
#merge two excel tables
table_c_excel = etl.join(blist_db_table_excel, expenses_db_table_excel)
table_c_excel.look()

In [ ]:
#rename column in excel table in accordance to the previous tables
table_d_excel = etl.rename(table_c_excel, 'branch_name','branch')
table_d_excel.look()

In [ ]:
#already edited csv file data to etl table for manipulation
db_table_csv=etl.fromcsv('data/csv_product_data.csv')

db_table_csv.look()

In [ ]:
#already edited json file data to etl table for manipulation
db_table_json = etl.fromcsv('json_product_data.csv')
db_table_json.look()

In [ ]:
#join json table and excel table
final_table_json=etl.lookupjoin(db_table_json, table_d_excel, key='branch')
final_table_json.look()

In [ ]:
#join csv table and excel table
final_table_csv=etl.lookupjoin(db_table_csv, table_d_excel, key='branch')
final_table_csv.look()

In [ ]:
#concatenation of json and csv table
final_unified_table= etl.cat(final_table_json, final_table_csv)

In [ ]:
final_unified_table= etl.leftjoin(final_unified_table, product_list, key='product')
final_unified_table.look()

In [ ]:
#cleaning of column fields
final_transformed_data_df= etl.addfield(final_unified_table, 'q', lambda cell:['total_quantity_purchased']+cell['quantity_purchased']+cell['total_quantity']+cell['quantity'])


In [ ]:
#adding fields to goods column
final_transformed_data_df=etl.addfield(final_transformed_data_df, 'goods', lambda cell:['product']+cell['item']+cell['sku'])

In [ ]:
#table cleaning of columns
table_clean= etl.cutout(final_transformed_data_df, 'total_quantity_purchased', 'quantity_purchased', 'product', 'sku' )




In [ ]:
#Check before final export
table_clean.look()

In [ ]:
#export etl table to CSV for filtering and dash 
etl.tocsv(table_clean, 'final_cleaned_df.csv')